In [73]:
import requests

In [74]:
from bedrock.client.client import BedrockAPI

In [75]:
SERVER = "http://localhost:81/"

In [76]:
api = BedrockAPI(SERVER)

In [77]:
filepath =  '/home/atam6/git/bedrock-core/examples/RAND2011study/Rand2011PNAS_cooperation_data.csv'

In [78]:
ingest_id = 'opals.spreadsheet.Spreadsheet.Spreadsheet'
resp = api.put_source('Rand2001',ingest_id, 'default',{'file':open(filepath, "rb")})
if resp.status_code == 201:
    print 'Source {0} successfully uploaded'.format(filepath)
else:
    print 'Failed to update {0}'.format(filepath)

INFO: 2017-09-06 11:36:06,428: putting source to: http://localhost:81/dataloader/sources/Rand2001/opals.spreadsheet.Spreadsheet.Spreadsheet/default/


Failed to update /home/atam6/git/bedrock-core/examples/RAND2011study/Rand2011PNAS_cooperation_data.csv


In [79]:
available_sources = api.list("dataloader","sources").json()
source = (s for s in available_sources if s["name"] == "Rand2001").next()
source_id = source["src_id"]
endpoint = api.endpoint("dataloader","sources/%s/explore/" % source_id)
resp = requests.get(endpoint)

INFO: 2017-09-06 11:36:06,663: Listing: http://localhost:81/dataloader/sources/


In [80]:
resp = api.create_matrix(source_id, 'rand_mtx')
mtx = resp[0]
matrix_id = mtx['id']
print(mtx)
#resp

{u'src_id': u'2876d3ecda384e6ca195fefc0de078e9', u'name': u'rand_mtx', u'created': u'2017-09-06 15:36:07.252085', u'outputs': [u'features_original.txt', u'features.txt', u'matrix.csv'], u'id': u'bb7f7692ed4b4f49aa5b8ecd50faf965', u'rootdir': u'/opt/bedrock/dataloader/data/2876d3ecda384e6ca195fefc0de078e9/bb7f7692ed4b4f49aa5b8ecd50faf965/', u'filters': {u'fluid_dummy': {}, u'previous_decision': {}, u'group_size': {}, u'decision0d1c': {}, u'playerid': {}, u'_Icondition_4': {}, u'_Icondition_3': {}, u'_Icondition_2': {}, u'sessionnum': {}, u'num_neighbors': {}, u'round_num': {}, u'condition': {}}, u'mat_type': u'csv'}


In [81]:
resp = api.analytic('opals.stan.Stan.Stan_GLM')
if resp.json():
    print("Stan_GLM Opal Installed!")
else:
    print("Stan_GLM Opal Not Installed!")

Stan_GLM Opal Installed!


In [82]:
resp

<Response [200]>

## Run Stan_GLM

In [83]:
analytic_id = "opals.stan.Stan.Stan_GLM"
inputData = {
    'matrix.csv': mtx,
    'features.txt': mtx
}

paramsData = [
            { "name" : "Formula", "attrname" : "formula", "value" : "decision0d1c ~ condition", "type" : "input" },
            { "name" : "GLM family", "attrname" : "family", "value" : 'binomial(link = "logit")', "type" : "input" },
            { "name" : "chains", "attrname" : "chains" , "value" : "3", "type" : "input"},
            { "name" : "iter", "attrname" : "iter" , "value" : "3000", "type" : "input"},
            { "name" : "prior", "attrname" : "prior" , "value" : "", "type" : "input"},
            { "name" : "prior_intercept", "attrname" : "prior_intercept" , "value" : "", "type" : "input"}
        ]

result_mtx = api.run_analytic(analytic_id, mtx, 'rand_stan_test', input_data=inputData, parameter_data=paramsData)

In [84]:
print result_mtx

{u'src_id': u'bb7f7692ed4b4f49aa5b8ecd50faf965', u'name': u'rand_stan_test', u'parameters': [{u'attrname': u'formula', u'type': u'input', u'name': u'Formula', u'value': u'decision0d1c ~ condition'}, {u'attrname': u'family', u'type': u'input', u'name': u'GLM family', u'value': u'binomial(link = "logit")'}, {u'attrname': u'chains', u'type': u'input', u'name': u'chains', u'value': u'3'}, {u'attrname': u'iter', u'type': u'input', u'name': u'iter', u'value': u'3000'}, {u'attrname': u'prior', u'type': u'input', u'name': u'prior', u'value': u''}, {u'attrname': u'prior_intercept', u'type': u'input', u'name': u'prior_intercept', u'value': u''}], u'created': u'2017-09-06 15:36:55.435707', u'outputs': [u'prior_summary.txt', u'summary.txt'], u'analytic_id': u'opals.stan.Stan.Stan_GLM', u'rootdir': u'/opt/bedrock/analytics/data/bb7f7692ed4b4f49aa5b8ecd50faf965/363e498a782e4c70b9cf16c07c1ab2fc/', u'id': u'363e498a782e4c70b9cf16c07c1ab2fc'}


## Get Stan_GLM outputs

In [85]:
result_mtx['src_id']

u'bb7f7692ed4b4f49aa5b8ecd50faf965'

In [86]:
result_mtx['id']

u'363e498a782e4c70b9cf16c07c1ab2fc'

In [87]:
import requests

In [88]:
base_url = "http://localhost:81/analytics/results/{0}/{1}/download/prior_summary.txt/prior_summary.txt/"

In [89]:
url = base_url.format(result_mtx['src_id'],result_mtx['id'])

In [90]:
response = requests.get(url)

In [91]:
print response.text

Priors for model 'output' 
------
Intercept (after predictors centered)
 ~ normal(location = 0, scale = 10)

Coefficients
 ~ normal(location = [0,0,0], scale = [2.5,2.5,2.5])
------
See help('prior_summary.stanreg') for more details


In [92]:
base_url = "http://localhost:81/analytics/results/{0}/{1}/download/summary.txt/summary.txt/"

In [93]:
url = base_url.format(result_mtx['src_id'],result_mtx['id'])

In [94]:
response2 = requests.get(url)

In [95]:
print response2.text


Model Info:

 function:  stan_glm
 family:    binomial [logit]
 formula:   decision0d1c ~ condition
 algorithm: sampling
 priors:    see help('prior_summary')
 sample:    4500 (posterior sample size)
 num obs:   3876

Estimates:
                   mean    sd      2.5%    25%     50%     75%     97.5%
(Intercept)          0.4     0.1     0.3     0.4     0.4     0.5     0.5
conditionRandom     -0.4     0.1    -0.6    -0.5    -0.4    -0.3    -0.2
conditionStatic     -0.5     0.1    -0.7    -0.6    -0.5    -0.4    -0.3
conditionViscous    -0.2     0.1    -0.4    -0.3    -0.2    -0.2     0.0
mean_PPD             0.5     0.0     0.5     0.5     0.5     0.5     0.6
log-posterior    -2668.6     1.4 -2672.1 -2669.3 -2668.3 -2667.6 -2666.9

Diagnostics:
                 mcse Rhat n_eff
(Intercept)      0.0  1.0  2940 
conditionRandom  0.0  1.0  3078 
conditionStatic  0.0  1.0  3255 
conditionViscous 0.0  1.0  3255 
mean_PPD         0.0  1.0  4500 
log-posterior    0.0  1.0  2041 

For each para